In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 10 17:34:18 2020

@author: Guillermo Sánchez Gutiérrez-Cabello
"""
#%%
import numpy as np
import pandas as pd
import os
import sklearn
#pd.options.mode.chained_assignment = None  # default='warn'


from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

import nltk
nltk.download("popular") # required to download the stopwords lists
from nltk.corpus import stopwords

spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')
#%%
mainpath = "C:/Users/guill/Documents/Universidad/PlataformaRefugiados/NAUTIA/DesarrolloPy/DataSetOriginales"

def fixBibliography(df):
    df = dfFix(df,"GENERAL INFORMATION - COUNTRY LEVEL")
    df.columns = ['GeneralInfo', 'CommunityCountry', 'RefugeeCountry']
    df.set_index('GeneralInfo', inplace = True)
    df = df.transpose()
    df = df.reset_index()
    return df

def dfFix(df,col1 = False,col2 = False):
    result = df.copy()
    if(col1):
        x = result.columns.get_loc(col1)
        result.drop(result.columns[0:x],axis = 1, inplace = True)
    if(col2):
        y = result.columns.get_loc(col2)
        result.drop(result.columns[y:],axis = 1, inplace = True)
    return result

def validColumn(cad):
    result = False
    if(cad == "index"):#Population
        result = True
    else:
        if(cad == "Type_of_settlement"):
            result = True
        else:
            if(cad == "General:settlement"):
                result = True
            else:
                if(cad == "general_info:_1_1_Choose_the_settlement"):
                    result = True
                else:
                    if(cad == "General_Information:Type_of_setlement"):
                        result = True
                    else:
                        if(cad == "General:Settlement"):
                            result = True
                        else:
                            if(cad == "Type_of_setlement"):
                                result = True
    return result

def dropRow(df,i):
    return df.drop(index = i)

def get_communityRows(df,cad,communityType): #la función pd.loc[] tiene un bug indiscriminado (https://github.com/pandas-dev/pandas/issues/8555)
    result = df
    if(communityType == 0):
        comm = "host_community"
    else:
        comm = "refugee_camp"
    for index, row in df.iterrows():
        if(row[cad] != comm):
            result = dropRow(result,index)
    return result

def setDataByIndex(df,communityType):
    array = df.columns
    i = 0
    df = df.replace("refugee","refugee_camp")
    df = df.replace("host_comunity","host_community")
    df = df.replace("RefugeeCountry","refugee_camp")
    df = df.replace("CommunityCountry","host_community")
    while(validColumn(array[i]) == False):
        i += 1
    df = get_communityRows(df,array[i],communityType)
    return df

def set_AllCSVtoDF(communityType):
    Bibliography = pd.read_excel(getPath(mainpath,"Bibliography_120220.xlsx"))
    Bibliography = fixBibliography(Bibliography)
    Bibliography = setDataByIndex(Bibliography,communityType)
    Entities = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Entities_Interview_results.csv")),communityType)
    LocalLeaders = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Local_leaders_v3_results.csv")),communityType)
    HouseHold = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Survey_household_v6_results.csv")),communityType)
    WomenGroup = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Women_Focus_Group2_results.csv")),communityType)
    SanitationInfra = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv")),communityType)
    Priorities = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Priorities_v3_results.csv")),communityType)
    GeneralForm = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_form_v3_results.csv")),communityType)
    PublicSpace = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Public_Space_results.csv")),communityType)
    WaterInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Water_Infrastructure_results.csv")),communityType)
    SanitationInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv")),communityType)
    WasteManagementInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Waste_Management_Infrastructure_results.csv")),communityType)
    EnergyINF = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Energy_Infrastructure_results.csv")),communityType)
    Business = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA1_0_Business_surveys_v3_results.csv")),communityType)
    MobilityINF = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0__Transport_servicesaccess_points_results.csv")),communityType) 
    ComunalServices = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Communal_Services_results.csv")),communityType) 
    GeneralCitizen = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_Citizen_Focus_Group_results.csv")),communityType)
    Shelter = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Shelter_results.csv")),communityType)
    FarmyardCrop = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Farmyard_and_Crops_results.csv")),communityType)
    return Bibliography,Entities,LocalLeaders,HouseHold,WomenGroup,SanitationInfra,Priorities,GeneralForm,PublicSpace,WaterInf,EnergyINF,SanitationInf,WasteManagementInf,EnergyINF,Business,MobilityINF,ComunalServices,GeneralCitizen,Shelter,FarmyardCrop

def concatDF(df1,df2):
    return  pd.concat([df1,df2],axis = 1, ignore_index = True, sort = True)

def mkCSV(df,fileName):
    df = df.dropna(how = 'all')
    df *= 1   
    fileName = fileName.lower()
    df.to_csv('DataSetFinales/'+fileName,sep=',',header = False, index=False, encoding='utf-8')
    
def getPath(mainpath,filename):
    return os.path.join(mainpath, filename)

def getSubColumnNames(df,x):
    columns = df.columns
    array = []
    for column in columns:
        column = column[x:]
        array.append(column)
    return pd.DataFrame(array) 

def addInstitutionAndType(df,array1,array2,instType):
    df = df.dropna(axis = 1)
    df = np.array(df)
    for row in df:
        for elem in row:
            array1 = np.append(array1,elem)
            array2 = np.append(array2,instType)
    return array1,array2

def politicalActor(df1,df2,df3,df4,df5):
    institution = []
    instType = []

    institution, instType  = addInstitutionAndType(df1,institution,instType,'Public Institution')
    institution, instType  = addInstitutionAndType(df2,institution,instType,'Private Institution')
    institution, instType  = addInstitutionAndType(df3,institution,instType,'NGO')
    institution, instType  = addInstitutionAndType(df4,institution,instType,'International Agency')
    institution, instType  = addInstitutionAndType(df5,institution,instType,'Local')

    institution = pd.DataFrame(institution)
    institution = institution.reset_index(drop = True)
    instType = pd.DataFrame(instType)
    instType = instType.reset_index(drop = True)
    
    return concatDF(institution,instType)


def get_claveValor(df1,df2,df3):
    array1 = np.array(df2)
    array2 = np.array(df1)
    result1 = []
    result2 = []
    result3 = []
    i = 0
    for row in array1:
        var = array2[i]
        for elem in row:
            result1 = np.append(result1,elem)
            result2 = np.append(result2,var)
        for r in np.array(df3):
            for l in r:
                result3 = np.append(result3,l)
        i+=1
    result2 = pd.DataFrame(result2)
    result2 = result2.reset_index(drop = True)
    result1 = pd.DataFrame(result1)
    result1 = result1.reset_index(drop = True)
    result3 = pd.DataFrame(result3)
    result3 = result3.reset_index(drop = True)
    return concatDF(result2,concatDF(result1,result3))

def get_number(df,df2):
    df = np.array(df)
    df2 = np.array(df2)
    array = np.array([])
    array2 = np.array([])
    for row in df:
        for elem in row:
            array = np.append(array,elem)
        for row2 in df2:
            for elem2 in row2:
                array2 = np.append(array2,elem2)
    array = (pd.DataFrame(array)).fillna(0) 
    array2 = pd.DataFrame(array2)
    return concatDF(array,array2)

def get_valueBySector(df1,df2):
    df2 = df2.reset_index()
    array1 = np.array(df1)
    i = 0
    for row in array1:
        for elem in row:
            if(elem == False):
                df2 = dropRow(df2,i)
        i += 1
    df2 = df2.set_index('index')
    return df2


def separateValues(df):
    if(df.isnull().values.all(axis=0)):
        result = df
    else:
        array = np.array(df)
        count_vectorizer = CountVectorizer(stop_words = spanish_stopwords+english_stopwords)
        corpus = []
        for row in array:
            for elem in row:
                corpus = np.append(corpus,[elem])
        X = count_vectorizer.fit_transform(corpus)
        array = count_vectorizer.get_feature_names()
        result = pd.DataFrame(array)
    return result

def vectorizeValue(df):
    df = df.dropna(how = 'all')
    season = pd.DataFrame()
    year = ['january','february','march','april','may','june','july','august','september','october','november','december']
    for row in np.array(df):
        corpus = np.array([])
        count_vectorizer = CountVectorizer(stop_words = spanish_stopwords+english_stopwords)
        for elem in row:
            corpus = np.append(corpus,[elem])
        X = count_vectorizer.fit_transform(corpus)
        array = count_vectorizer.get_feature_names()
        season = concatDF(season,pd.DataFrame(array))
    season = season.T
    result = pd.DataFrame()
    for row in np.array(season):
        vect = np.array([],dtype = bool)
        for elem in year:
            flag = False
            for month in row:
                if(month == elem):
                    flag = True
            if(flag):
                vect = np.append(vect,True)
            else:
                vect = np.append(vect,False)
        result = concatDF(result, pd.DataFrame(vect))
    return result.T



def set_sector(df,sect, concat = True):
    sector = np.array([])
    df = df.dropna(how = 'all')
    df = np.array(df)
    for column in df:
        sector = np.append(sector,sect)
    sector = pd.DataFrame(sector)
    df = pd.DataFrame(df)
    if(concat):
        result = concatDF(sector,df)
    else:
        result = sector
    return result 

def dropNaAndResetIndex(df):
    df = df.dropna(how = 'all')
    df = np.array(df)
    return pd.DataFrame(df)
    

def get_applianceDF(df):
    df1 = dfFix(df,0,1)
    df2 = dfFix(df,1)
    array = np.array([])
    appliance = np.array([])
    for row in np.array(df1):
        corpus = np.array([])
        for elem in row:
            corpus = np.append(corpus,[elem])
        count_vectorizer = CountVectorizer(stop_words = spanish_stopwords+english_stopwords, vocabulary = ['light_bulbs','mobile_phone','radio_music_pl','tv_dvd','laptop_tablet_','fridge','electrical_sto','others'])
        X = count_vectorizer.fit_transform(corpus)
        array = count_vectorizer.get_feature_names()
        for elem in array:
            appliance = np.append(appliance,elem)
    appliance = pd.DataFrame(appliance)
    hours = np.array([])
    for row in np.array(df2):
        for elem in row:
            hours = np.append(hours,elem)
    hours = pd.DataFrame(hours)
    hours = (hours.dropna())
    return concatDF(appliance,hours)
    

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [2]:
Bibliography,Entities,LocalLeaders,HouseHold,WomenGroup,SanitationInfra,Priorities,GeneralForm,PublicSpace,WaterInf,EnergyINF,SanitationInf,WasteManagementInf,EnergyINF,Business,MobilityINF,ComunalServices,GeneralCitizen,Shelter,FarmyardCrop = set_AllCSVtoDF(0)

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
Bibliography

GeneralInfo,index,Population,Country's name,Number of inhabitants (#),Mujeres menores de 5 años (%),Hombres menores de 5 años (%),Mujeres entre 5-17 años (%),Hombres Entre 5-17 años (%),Mujeres entre 18-59 años (%),Hombres entre 18-59 años (%),Mujeres mayores de 60 años (%),Hombres mayores de 60 años (%),Total population,Total children younger than 5 years (%),Total population between 5-17 years (%),Total population between 18 to 59 years (%),Total population 60 años (%),Growth rate of populatoin (%),Refugee population (%),Human development index (points),Life expentancy at birth (years),Culture,Ethnic groups,Ethnich group 1,Ethnich group 2,Ethnich group 3,Religion,Religion 1,Religion 2,Religion 3,Language,Language 1,Language 2,Language 3,Economy and well-being,Population with access to employment,Agriculture (%),Livestock (%),Industry (%),Services (%),Poor people rate (%),Index GINI (points),PIB per cap (USD),Poverty line (USD/day),Local currency,Exchange rate (local currency vs USD),Government,System of governance (choose one),Parliamentary republic,Presidential republic,Syngle-party republic,Parliamentary monarchy,Absolute monarchy,Dictatorship,Another,Territorial and Urbanistic,Location,Urban population (%),Rural population (%),Population density,Urban (inhabitants/hectares),Rural (inhabitants/hectares),Infrastructures,Access to drinking water supply,Total (%),Rural agua (%),Urbano agua (%),Access to improved sanitation,Total (%),Rural saneamiento(%),Urbano saneamiento (%),Access to electricity,Total (%),Rural electricidad (%),Urbano electricidad (%),Local electrificy tariff (local currency/kWh),Matrix of electricity generation,Hydropower (%),Diesel generation (%),Gas (%),Coal (%),Solar photovoltaic (%),Wind power (%),Biomass/biofuels (%),High voltage (kV),Low voltage (V),Access to Services,Illiteracy rate (%),Internet access rate (%),Shelter,Slum population rate (%),SPECIFIC INFORMATION - SETTLEMENTS LEVEL,CULTURE IDENTIFICATION,SETTLEMENTS Religion,SETTLEMENTS Religion 1,SETTLEMENTS Religion 2,SETTLEMENTS Religion 3,SETTLEMENTS Language,SETTLEMENTS Language 1,SETTLEMENTS Language 2,SETTLEMENTS Language 3,PHYSICAL AND ENVIRONMENT DATA,Location,Latitude,Longitud,Topography,Upper bound (m),Lower bound (m),FOOD SECURITY,Cause of food insecurity,Cause 1,Cause 2,Cause 3,Affected groups due to food insecurity,Children,Women,Men,Old people,Calories of the typical dish,Pork (200 kcal/100g),Beef 180 kcal/100 g,Chicken: 130 kcal/100 g,Lamp: 125 kcal/100 g,Cereales/grains: 350 kcal/100 g,Legumes: 350 kcal/100 g,Fruits: 50-200 kcal/100 g,Intake (g) - default value 70g-,GENERAL INFORMATION OF REFUGEES SETTLEMENT,Implementation date of the refugee camp (year),Migration reasons,Reason 1,Reason 2,Reason 3,Reason 4,Climate,"Tropical (Write one: Af, Aw or Am)","Dry (Write one: Bsh, Bsk, Bwh or Bwk)","Temperated (Write one: Cdb, Cfc, Csa, Csb, Cfa, Cwa, Cwb)","Continental (Write one: Dfa, Bwa, Dsa or Dfb, Dwb, Dsb)",Temperature,Max (ºC),Min (ºC),Average (ºC),Relative humidity (%),Annual precipitation,Max (mm),Min (mm),Solar irradiance (kW/h),Wind speed km/h,Additional information,Heights of close rivers (r=20km),r.1,r.2,r.3,r.4,ACTORS (PARTNERS) IDENTIFICATION,Public institutions,pu.1,pu.2,pu.3,pu.4,pu.5,pu.6,pu.7,Private institutions,pr.1,pr.2,pr.3,pr.4,pr.5,pr.6,Non-profit organizations/NGOs,np.1,np.2,np.3,np.4,np.5,np.6,International cooperation agencies,int.1,int.2,int.3,int.4,int.5,int.6,Local representatives/local committees/ local liders,ld.1,ld.2,ld.3,ld.4,ld.5,ld.6
0,host_community,Host community's country,Etiopia,NaN,8,12,15,25,20,10,6,4,NaN,20,40,30,10,2.85,0.86,0.448,64.6,NaN,NaN,Oromo,Amara,Tiigray,NaN,Islam,Cristianismo Ortodoxo,Protestantismo,NaN,Trigrinya,Árabe,NaN,NaN,79.41,40,35,5,20,23.5,39.1,767,1.25,Birr,"1 ETB = 0,0363915 USD",NaN,NaN,Si,No,No,No,No,No,No,NaN,NaN,20.7,79.3,NaN,7000,2900,NaN,NaN,NaN,66.0,95.0,NaN,NaN,NaN,NaN,NaN,57.0,26.5,85.4,residential > 500 kWh = 0.6943 birr + service ...,NaN,92.0,1.0,0.0,0.0,0.0,7.

In [5]:
Entities

,start,end,Type_of_settlement,Type_Host_Community,Identification:Institution_Name,Identification:Contact_person,Identification:Work_Area,GENERAL_INFORMATION:Secondary_movement,GENERAL_INFORMATION:Relationship,GENERAL_INFORMATION:Movement_outside,GENERAL_INFORMATION:Distance,Population:Women:Infants,Population:Women:Childs,Population:Women:Adults,Population:Women:Olders,Population:Men:Infants_001,Population:Men:Childs_001,Population:Men:Adults_001,Population:Men:Olders_001,Fuel_Cost:Main_Fuel,Fuel_Cost:Fuel_Cost_Diesel,Fuel_Cost:Fuel_Cost_Kerosene,Fuel_Cost:Fuel_Cost_Ethanol,Fuel_Cost:Fuel_Cost_Gas_cylder,ENERGY:Electricity_network,ENERGY:Covered_services,ENERGY:Power_failure,ENERGY:Demand_Curve,ENERGY:Street_Light,Urban_Planning_001:Urban_Planning,Urban_Planning_001:Period,Urban_Planning_001:Growth_area,Urban_Planning_001:Forecast,Urban_Planning_001:Land_Managment,Urban_Planning_001:Risk_Managment,Urban_Planning_001:Risk_Identified,Shelter:Plot_Area,Shelter:Plot_Limits,Shelter:Housing_Improvement,Shelter:Total_shelter,Shelter:Cost,Shelter:Shelter_Construction,Shelter:Vulnerable_Area:Vunerable_Area,Enviormental_Issues:Risk:Risk_Flood,Enviormental_Issues:Risk:Risk_dryness,Enviormental_Issues:Risk:Risk_earthquake,Enviormental_Issues:Risk:Risk_storm,Enviormental_Issues:Risk:Risk_plage,Enviormental_Issues:Deforestation,Enviormental_Issues:High_enviormental_value,Enviormental_Issues:Native_Plant,Enviormental_Issues:Native_Crops,Water_table,Sanitation:Personal_hygiene,Sanitation:Excreta,Sanitation:Open_defecation,Sanitation:Type_of_Latrine,Sanitation:Individual_Latrines,Waste_Managment:Waste_Collection,Waste_Managment:Landfill_Details:Location_1,Waste_Managment:Landfill_Details:Location_2,Waste_Managment:Landfill_Details:Adecuate_land,Water:Quality,Water:Treatment,Water:Comsuption,Water:Distribution,Water:Time,Women_Patrol,Education_Issues,Data_Access,Antenna,meta:instanceID
1,2020-03-26 15:22:34.874,2020-03-26 15:25:38.92,host_community,hostcommunityA,ZOA,Michael,WASH food_security energy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:8ce964c6-d7bf-4e7c-a010-94276aa6e79a
3,2020-03-26 15:55:29.364,2020-03-26 16:02:29.146,host_community,hostcommunityA,NRC,Acting manager area,manager_entity food_security,NaN,NaN,NaN,NaN,2000.0,3000.0,2000.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,no,NaN,goverment,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,1.0,far_away,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,economic_issue,NaN,2.0,uuid:52dfe799-cd79-42fe-abab-f5db82b8f441


In [6]:
LocalLeaders.shape

(5, 22)

In [7]:
HouseHold.shape

(5, 42)

In [8]:
WomenGroup.shape

(3, 27)

In [9]:
SanitationInfra.shape

(6, 14)

In [10]:
Priorities

,start,end,Type_of_setlement,Type_of_Host_Community,Participants,Participants_001,Participants_002,group_yf0yl72:Instruction_1,group_yf0yl72:Energy_1,group_yf0yl72:Shelter_1,group_yf0yl72:Water_Access_1,group_yf0yl72:Sanitation_1,group_yf0yl72:Education_1,group_yf0yl72:Health_1,group_yf0yl72:Public_space_1,group_yf0yl72:Food_1,group_yf0yl72:TICS_1,group_yf0yl72:Work_1,group_yf0yl72:Waste_Managment,group_yf0yl72:Public_Transport_001,group_yf0yl72:Religious_Center,group_yf0yl72:Socio_Cultural_Center,group_yf0yl72:Market,Priority_2:Instruction,Priority_2:Energy_2,Priority_2:Shelter_2,Priority_2:Water_Access_2,Priority_2:Sanitation_2,Priority_2:Education_2,Priority_2:Health_2,Priority_2:Public_space_2,Priority_2:Food_2,Priority_2:TICS_2,Priority_2:Work_2,Priority_2:Waste_Managment_2,Priority_2:Public_Transport_2,Priority_2:Religious_Center_2,Priority_2:Socio_Cultural_Center_2,Priority_2:Market_2,Priority_3:Instruction_001,Priority_3:Energy_3,Priority_3:Shelter_3,Priority_3:Water_Access_3,Priority_3:Sanitation_3,Priority_3:Education_3,Priority_3:Health_3,Priority_3:public_space_3,Priority_3:Food_3,Priority_3:TICS_3,Priority_3:Work_3,Priority_3:Waste_Managment_3,Priority_3:Public_Transport,Priority_3:Socio_Cultural_Center_3_001,Priority_3:Religious_Center_3_001,Priority_3:Market_001,Priority_4:Instruction_002,Priority_4:Energy_4_001,Priority_4:Shelter_4_001,Priority_4:Water_Access_4_001,Priority_4:Sanitation_4_001,Priority_4:Education_4_001,Priority_4:Health_4_001,Priority_4:public_space_4,Priority_4:Food_4,Priority_4:TICS_4,Priority_4:Work_4,Priority_4:Market_4,Priority_4:Religious_Center_4_001,Priority_4:Socio_Cultural_Center_4_001,Priority_4:Public_Transport_4,Priority_4:Waste_Managment_4,Priority_5:Instruction_003,Priority_5:Energy_4,Priority_5:Shelter_4,Priority_5:Water_Access_4,Priority_5:Sanitation_4,Priority_5:Education_4,Priority_5:Health_4,Priority_5:public_space_4_001,Priority_5:Food_4_001,Priority_5:TICS_5,Priority_5:Work_5,Priority_5:Market_5,Priority_5:Religious_Center_5,Priority_5:Socio_Cultural_Center_5,Priority_5:Public_Transport_5,Priority_5:Waste_Managment_5,meta:instanceID
2,2020-03-24 13:29:57.31,2020-03-24 13:34:05.013,host_community,hostcommunityA,NaN,50.0,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,5.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,10.0,NaN,2.0,NaN,3.0,NaN,2.0,NaN,4.0,NaN,2.0,NaN,3.0,NaN,2.0,NaN,2.0,NaN,10.0,NaN,2.0,NaN,10.0,NaN,2.0,NaN,NaN,2.0,2.0,NaN,2.0,NaN,15.0,NaN,2.0,NaN,2.0,NaN,1.0,NaN,1.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,0.0,uuid:5d576926-df41-45fa-b0fd-d848f2d38e95
3,2020-03-24 13:34:10.515,2020-03-24 13:35:20.183,host_community,hostcommunityA,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:e6f9e3e9-864b-497e-8ac7-aaa3a6305197
4,2020-03-24 13:35:27.751,2020-03-24 13:35:54.85,host_community,hostcommunityA,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:f63f5642-a482-442d-b481-7946af92ba26


In [11]:
GeneralForm.shape

(1, 19)

In [12]:
PublicSpace.shape

(3, 14)

In [13]:
WaterInf.shape

(4, 13)

In [14]:
SanitationInf.shape

(6, 14)

In [15]:
WasteManagementInf.shape

(2, 10)

In [16]:
EnergyINF.shape

(8, 38)

In [17]:
Business.shape

(5, 25)

In [18]:
MobilityINF.shape

(2, 10)

In [19]:
ComunalServices.shape

(40, 63)

In [20]:
GeneralCitizen.shape

(3, 43)

In [21]:
Shelter.shape

(5, 27)

In [22]:
FarmyardCrop.shape

(6, 18)

In [23]:
PublicSpace.shape

(3, 14)

In [24]:
def set_defaultColumn(df):
    default = np.array([])
    for row in df:
        default = np.append(default,np.nan)
    default = pd.DataFrame(default)
    return concatDF(df,default)        
        
def replacestr(df,cad1,cad2):
    cols=list(df.columns)
    for col in cols:
        df[col] = df[col].astype(str).str.replace(cad1,cad2)
    return df


In [27]:
S_BuildingQuality = dfFix(ComunalServices,"Construction_Details:Appropiate_Roof","meta:instanceID") 
S_BuildingQuality = S_BuildingQuality.isin(["yes"])
mkCSV(S_BuildingQuality,"S_BuildingQuality.csv")

In [28]:
S_BuildingQuality

,Construction_Details:Appropiate_Roof,Construction_Details:Structural_Security,Construction_Details:Resilient,Construction_Details:Thermal_Confort
19,True,True,True,True
20,False,False,False,False
21,True,True,True,True
24,True,False,False,False
26,True,False,False,True
27,True,True,True,True
29,False,True,False,False
32,False,False,False,False
33,False,False,False,False
34,False,False,False,False
